# Project Phase 4 Appendix: Data Cleaning Description
### Samantha Rubin, Hanwen Lin, Liz Espinoza, and Hanna Kang

Below, we import all the needed packages as well as the initial datasets:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

raw_covid = pd.read_csv("jrc-covid-19-world-latest.csv")
gini_data = pd.read_csv("gini_data2.csv")
gdp_data = pd.read_csv("gdp_data.csv")
pop_data = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_2106202.csv")
pop_den = pd.read_csv("pop_density.csv")
unemploy_rates = pd.read_csv("unemployment_rates.csv")

For gdp_data and gini_data, what should be the column labels are actually stored in the third row, and the data starts on the fourth row. This is fixed below:

In [4]:
gdp_data.columns = gdp_data.loc[3]
gdp_data = gdp_data.drop(index = [0,1,2,3])

Below, we work on removing unnecessary columns and renaming them:

In [8]:
#making covid data column names lowercase
new_colnames = [i.lower() for i in raw_covid.columns]
covid_cases = raw_covid.copy()
for i in covid_cases.columns:
    covid_cases.columns = new_colnames

#dropping unnecessary columns:
covid_cases = covid_cases[['date', 'iso3', 'continent', 'countryname'
                           , 'cumulativepositive', 'cumulativedeceased'
                           , 'currentlypositive']]
#renaming columns
covid_cases.rename(columns = {'iso3':'country_code','countryname':'country_name'
                              , 'cumulativepositive':'cumulative_pos', 'cumulativedeceased':'cumulative_dec'
                              , 'currentlypositive':'curr_pos'}, inplace = True)



#clean gdp data, drop unecessary columns and rename them
new_colnames = []
gdp = gdp_data.copy()
gdp.rename(columns = {"Country Name":"country_name","Country Code":"country_code",2019:"gdp_per_cap"}, inplace = True)
gdp = gdp[["country_name","country_code","gdp_per_cap"]]

#drop NaNs in gdp
gdp = gdp.dropna(axis = 0,subset=["gdp_per_cap"])

#clean gini column names
gini_data.rename(columns = {"Country Code":"country_code",
                          "Country":"country","Value":"gini_index",'Year':"gini_year"},inplace = True)

#clean population data
pop_data.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2019":"population"}
                ,inplace = True)
pop_data = pop_data[["country_code",'country_name','population']]

#clean population density 
pop_den.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2018":"density"}
               ,inplace = True)
pop_den = pop_den[['country_code','country_name','density']]

#clean unemployment rates 
unemploy_rates.rename(columns = {"Country Code":"country_code","Country Name":"country_name","2019":"rates"}
                      ,inplace = True)
unemploy_rates = unemploy_rates[['country_code','country_name','rates']]

Below, we matched countries via country codes, only keeping the countries that had GDP, Gini Index, population density, and unemployment rate values. We printed the length to see how many countries were dropped:

In [11]:
#adding gdp 
covid_gdp = pd.merge(left = covid_cases, right = gdp, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp))

#adding gini
covid_gdp_gini = pd.merge(left = covid_gdp, right = gini_data, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding population 
covid_gdp_gini = pd.merge(left = covid_gdp_gini, right = pop_data, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding population density
covid_gdp_gini = pd.merge(left = covid_gdp_gini, right = pop_den, left_on = "country_code",right_on = "country_code")
print(len(covid_gdp_gini))

#adding unemployment rates
covid_data = pd.merge(left = covid_gdp_gini, right = unemploy_rates, left_on = "country_code",right_on = "country_code")
print(len(covid_data))

#removing duplicate columns and renaming
covid_data = covid_data.drop(columns = ["country",'country_name_y',"country_name"])
covid_data.rename(columns = {"country_name_x":"country_name"},inplace = True)
covid_data.rename(columns = {'density':'pop_density', 'rates': 'unemploy_rates'}, inplace = True)
covid_data.drop_duplicates(subset = 'country_name', keep = 'first', inplace = True)

166
140
140
136
136


Below we display the final cleaned dataset:

In [12]:
covid_data.head()

date country_code continent country_name  cumulative_pos  \
0  2021-03-13          ALB        EU      Albania          116821   
1  2021-02-15          DZA        AF      Algeria          110711   
2  2021-02-15          AGO        AF       Angola           20366   
3  2021-02-15          ARG        SA    Argentina         2029008   
4  2021-02-15          AUS        OC    Australia           28900   

   cumulative_dec   curr_pos   gdp_per_cap  gini_index  gini_year  \
0            2030    34970.0   5353.244856        29.0       2012   
1            2939   107772.0   3973.964072        27.6       2011   
2             492    19874.0   2790.726615        42.7       2008   
3           50327  1978681.0   9912.281809        41.2       2017   
4             909    27991.0  55060.326100        35.8       2014   

  country_name  population  pop_density unemploy_rates  
0      Albania   2854191.0  104.6122628    11.47000027  
1      Algeria  43053054.0  17.73006763    11.81000042  
2       Angola  31825295.0  24.71305206    6.929999828  
3    Argentina  44938712.0   16.2585101    9.840000153  
4    Australia  25364307.0   3.24787091    5.159999847